<a href="https://colab.research.google.com/github/Nattyb21/diss/blob/main/finalstartynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [520]:
!pip install mplsoccer

In [521]:
import numpy as np
import random
from mplsoccer import Pitch
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML
import math

import time
import csv

from matplotlib.lines import Line2D

Ideas to implement:


Passing Mechanics:

    - Add more pass types
    - Passes affected by playstyle (e.g. direct more likely to long pass to striker)
    - More realistic passes (less random and say they build up are likely to pass around the back more and make less forward passes)
    - Passes going to a player rather than random x amount of places (unless like a throughball)


Positional Mechanics:

    - Passes based off where players (going to pass to closer players)
    - Players move as the game goes on (e.g. player moves up pitch with ball, team will follow)


Bayesian Model:

    - Implement the Bayesian Model
    
Real Data:

    - Implement the real data
    - Alter team strength algorithm
    - Add more playstyles
    - Add more formations


Misc:

    - Visualise the data

In [522]:
team_strength = {
    "home": {"attacking": 8.9, "defensive": 7.8, "possession": 8.2},
    "away": {"attacking": 7.2, "defensive": 6.5, "possession": 6.5}
}


# GameState

In [523]:
class GameState:
  """
  The current state of the simulation
  This class handles the game logic, ball and player movement as well
  possession
  """

  pitch_width = 120
  pitch_height = 80

  def __init__(self):  # Replace all variables later with database
    """ Initialise the game state variables """

    self.time = 0
    self.scoreline = {"home":0,"away":0}
    self.possession = "home"
    self.team_style, self.team_approach, self.formation = self.team_setup()
    self.ball_x = 60  #120 long
    self.ball_y = 40  #80 wide
    self.team_strength = team_strength
    self.ball_location = "mid"  # will remove soon
    self.player_pos = self.initialise_formation()
    self.players = self.initialise_player()
    self.shot_data = []
    self.pass_data = []
    self.event_data = []
    self.store_starting_formation = []
    self.original_positions = self.initialise_formation()

  def store_starting_formation(self):
    self.starting_positions = self.initialise_formation()

  def team_setup(self):    #Later will come from real data
     team_style = {"home": "long ball", "away": "counter attack"}
     team_approach = {"home": "balanced", "away": "attacking"}
     formation = {"home": "4-3-3", "away": "4-3-3"}  # Can be expanded later
     return team_style, team_approach, formation

  def change_playstyle(self, team, new_playstyle = None, new_formation = None):

    if new_playstyle:
      if new_playstyle in ["long ball", "possession", "counter attack"]:
        self.team_style[team] = new_playstyle
      else:
        print(f"Invalid playstyle, Options available: long ball, possession, counter attack")

    if new_formation:
      if new_formation in ["4-3-3", "4-4-2", "3-4-3", "5-3-2", "4-2-3-1"]:
        self.formation[team] = new_formation
        self.adjust_new_positions (team, new_formation)
      else:
        print("Invalid formation, Options available: 4-3-3, 4-4-2, 3-4-3, 5-3-2, 4-2-3-1")

  def initialise_formation(self):
    #4-3-3 example for now
    return {
        "home" : {
             "GK": (5, 40),
             "LCB": (20, 30), "RCB": (20, 50), "LB": (15, 15), "RB": (15, 65),
             "LCM": (40, 30), "RCM": (40, 50), "CDM": (35, 40),
             "LW": (70, 15), "ST": (80, 40), "RW": (70, 65)
            },
            "away": {
                "GK": (115, 40),
                "LCB": (100, 30), "RCB": (100, 50), "LB": (105, 15), "RB": (105, 65),
                "LCM": (80, 30), "RCM": (80, 50), "CDM": (85, 40),
                "LW": (50, 15), "ST": (40, 40), "RW": (50, 65)
            }
        }


  def initialise_player(self):
    players = {"home": {}, "away": {}}

    for team in ["home", "away"]:
      for role, position in self.player_pos[team].items():
        x = position[0]
        if team == "away":
          x = 120 - x
        player =  PlayerMechanicsBayesian(role, position[0], position[1], role, team)
        players[team][role] = player
    return players


  def get_player_pos(self):
    positions = {"home":{},"away":{}}

    for team in ["home", "away"]:
      for role, player in self.players[team].items():
        positions[team][role] = (player.x_pos, player.y_pos)
    return positions

  def get_nearest_player(self, team, x, y):
    nearest_player = None
    min_distance = float('inf')

    for role, player in self.players[team].items():
      ply_x, ply_y = player.get_position()
      distance = np.sqrt((x - ply_x)**2 + (y - ply_y)**2)

      if distance < min_distance:
        min_distance = distance
        nearest_player = role
    return nearest_player

  def update_ball_location(self):
    nearest_team = self.possession
    nearest_role = self.get_nearest_player(nearest_team, self.ball_x, self.ball_y)
    nearest_player = self.players[nearest_team][nearest_role]
    self.ball_x, self.ball_y = nearest_player.get_position()
    #print(f"Ball Location Updated: Team={nearest_team}, Player={nearest_role}, Ball=({self.ball_x}, {self.ball_y})")

  def lose_possession(self):
    print(f"Before possession loss: Team in possession = {self.possession}, Ball at x = {self.ball_x}")

    if self.possession == "home":
      self.possession = "away"
      #print("Home team lost the ball")
    else:
      self.possession = "home"
     # print("Away team lost the ball")

 #   print(f"After possession loss: Team in possession = {self.possession}, Ball at x = {self.ball_x}")


  def restart_ball(self):
    if self.possession == "home":
      self.possession = "away"
    else:
      self.possession = "home"

    self.ball_x = 60
    self.ball_y = 40
    print(f"Ball restarted, x:{self.ball_x}, y: {self.ball_y}")


  def player_movement(self):
    for team in ["home", "away"]:
            for role, player in self.players[team].items():
                if isinstance(player, PlayerMechanics):
                    player.enhanced_pos_adjust(self.ball_x, self.ball_y, self.ball_location, self)
                else:
                    player.pos_adjust(self.ball_x, self.ball_y, self.ball_location)




  def adjust_new_positions(self, team, new_formation):
    formation_positions = {
            "4-3-3": {"GK": (5, 40), "LCB": (20, 30), "RCB": (20, 50), "LB": (15, 15), "RB": (15, 65),
                       "LCM": (40, 30), "RCM": (40, 50), "CDM": (35, 40), "LW": (70, 15), "ST": (80, 40), "RW": (70, 65)},
            "4-4-2": {"GK": (5, 40), "LCB": (20, 30), "RCB": (20, 50), "LB": (15, 15), "RB": (15, 65),
                       "LM": (50, 15), "CM1": (40, 30), "CM2": (40, 50), "RM": (50, 65), "ST1": (80, 35), "ST2": (80, 45)},
            "3-5-2": {"GK": (5, 40), "CB1": (25, 20), "CB2": (25, 40), "CB3": (25, 60), "LWB": (50, 10), "RWB": (50, 70),
                       "CM1": (40, 30), "CM2": (40, 50), "CDM": (35, 40), "ST1": (80, 35), "ST2": (80, 45)},
            "5-3-2": {"GK": (5, 40), "LCB": (15, 25), "CB": (15, 40), "RCB": (15, 55), "LB": (10, 10), "RB": (10, 70),
                       "CM1": (40, 30), "CM2": (40, 50), "CDM": (35, 40), "ST1": (80, 35), "ST2": (80, 45)}
        }
    if new_formation in formation_positions:
            self.formation[team] = new_formation
            for role, position in formation_positions[new_formation].items():
              x, y = position
              if team == "away":
                  x = 120 - x
              self.players[team][role].x_pos = x
              self.players[team][role].y_pos = y
              self.players[team][role].set_movement_zones()

    else:
      raise ValueError(f"Invalid formation: {new_formation}")


  def update_player_positions(self):
        """Updates player movement each tick."""
        for team, team_players in self.players.items():
            for role, player in team_players.items():
              if isinstance(player, PlayerMechanics):
                player.bayesian_position_adjust(self.ball_x, self.ball_y, self)
                decision = player.make_decision(self)
                if decision == 'shoot':
                    # Implement shooting logic
                    pass
                elif decision == 'dribble':
                    # Implement dribbling logic
                    pass
                elif decision[0] == 'aggressive_pass':
                    # Implement aggressive passing logic
                    pass
                elif decision == 'safe_pass':
                    # Implement safe passing logic
                    pass
                elif decision == 'tackle':
                    # Implement tackling logic
                    pass
                elif decision == 'position':
                  player.return_to_formation(self)

                player.make_intelligent_run(self)
              else:
                  player.pos_adjust(self)
                  player.ball_reaction(self)
                  player.position_change(self)


                #should this be here or in their classes???

  def update_pass_data(self, pass_type, distance, pressure, success, passer_x, passer_y, target_x, target_y):
        player_positions = self.get_player_pos()
        self.pass_data.append({
            'event_type':'pass',
            'time' : self.time,
            'player_positions': player_positions,
            'start': (passer_x, passer_y),
            'end': (target_x, target_y),
            'success': success
        })

  def update_event_data(self, event_type, start_x, start_y, end_x, end_y, shot_target=None):
        self.event_data.append({
            'event_type': event_type,
            'start': (start_x, start_y),
            'end': (end_x, end_y),
            'shot_target': shot_target
        })



# Player Class

In [524]:
class Player:
  def __init__(self, name, x_pos, y_pos, role,team, attributes=None):
    self.name = name
    self.x_pos = x_pos
    self.y_pos = y_pos
    self.role = role
    self.team = team
    self.base_x = x_pos
    self.base_y = y_pos
    #Default attributes if none found
    self.attributes = attributes if attributes else {
        "speed": random.uniform(5, 10),
        "stamina": random.uniform(5, 10),
        "agility": random.uniform(5, 10),
        "positioning": random.uniform(5, 10),
        "passing_skill": random.uniform(5, 10),
        "shooting": random.uniform(5, 10),
        "tackling": random.uniform(5, 10),
        "vision": random.uniform(5, 10),
        "acceleration": random.uniform(5, 10),
        "dribbling": random.uniform(5, 10)
        }

    self.fatigue = 1.0 #Full energy



  def get_position(self):
    return self.x_pos, self.y_pos

  def move_to(self, target_x, target_y, speed=None):
    if speed is None:
      speed = self.attributes['speed'] * self.fatigue

    distance = np.sqrt((target_x - self.x_pos) ** 2 + (target_y - self.y_pos) ** 2)

    if distance > speed:
      direction_x = (target_x - self.x_pos) / distance
      direction_y = (target_y - self.y_pos) / distance
      self.x_pos += direction_x * speed + random.uniform(-0.5,0.5)  #random adjustment
      self.y_pos += direction_y * speed + random.uniform(-0.5,0.5)
      self.reduct_fatigue(distance)
    else:
      self.x_pos, self.y_pos = target_x, target_y
      self.reduct_fatigue(distance)

  def reduct_fatigue(self, distance):
    fatigue_reduction = (distance / (self.attributes["stamina"] * 450))
    self.fatigue = max(0, self.fatigue - fatigue_reduction)

  def recover_fatigue(self, recovery_rate = 0.002):
    self.fatigue = min(1, self.fatigue + recovery_rate)

  def pos_adjust(self, game_state):
    ball_x, ball_y = game_state.ball_x, game_state.ball_y
    team_style = game_state.team_style[self.team]

    if self.team == "away":
            ball_x = 120 - ball_x  # Mirror ball position for away team

    if self.role in ["ST", "LW", "RW"]:
      if team_style == "counter attack":
        self.move_to(ball_x + 10, ball_y)
      elif team_style == "possession":
        self.move_to(ball_x - 5, ball_y)
      else:
        self.move_to(ball_x, ball_y + random.randint(-5, 5))
    elif self.role in ["LCM","CM1", "CDM", "CAM","CM2", "RCM"]:
      self.move_to(ball_x - 10, ball_y)
    elif self.role in ["RCB","LCB", "LB", "RB"]:
      if game_state.possession == self.team:
        self.move_to(ball_x - 20, ball_y)
      else:
        self.move_to(ball_x - 30, ball_y)

    self.recover_fatigue()


  def ball_reaction(self, game_state):
    ball_x, ball_y = game_state.ball_x, game_state.ball_y
    if abs(ball_x - self.x_pos) < 10 and abs(ball_y - self.y_pos) < 10:
      self.move_to(ball_x, ball_y)

# Player Mechanics

In [525]:
class PlayerMechanics(Player):
  def __init__(self, name, x_pos, y_pos, role,team, attributes = None):
    super().__init__(name, x_pos, y_pos, role, team)

    self.last_movement = 0
    self.move_cooldown = 0.5 #this is in seconds

    self.base_position = self.set_base_position()

    default_attributes = {
            'speed': 7,
            'positioning': 7,
            'passing_skill': 7,
            'shooting': 7,
            'tackling': 7,
            'vision': 7,
            'acceleration': 7,
            'agility': 7
        }
    if attributes:
      self.attributes.update(attributes)  # Update with provided attributes
    else:
      self.attributes.update(default_attributes) # Use defaults if none provided

        # Track player fatigue (0-100, 100 being fresh)
    self.energy = 100

        # Define movement zones based on role
    self.set_movement_zones()

    self.position_history = [(self.x_pos, self.y_pos)]

  def set_base_position(self):
    base_pos = {
        'attack': (self.x_pos, self.y_pos),
        'defense': (self.x_pos, self.y_pos),
        'transition':(self.x_pos, self.y_pos)
    }

    if self.role in ['ST', 'LW', 'RW']:
            base_pos['attack'] = (self.x_pos + 10, self.y_pos)
            base_pos['defense'] = (self.x_pos - 20, self.y_pos)
    elif self.role in ['LCM', 'RCM', 'CDM']:
            base_pos['attack'] = (self.x_pos + 5, self.y_pos)
            base_pos['defense'] = (self.x_pos - 10, self.y_pos)

    return base_pos

  def det_phase (self, game_state):
    if game_state.possession == self.team:
      return 'attack'
    elif abs (game_state.ball_x - self.x_pos) < 30:
      return 'transition'
    else:
      return 'defense'

  def set_movement_zones(self):
        """Define movement zones based on player role"""
        # Default movement zones (x_min, x_max, y_min, y_max)
        if self.team == "home":
            if self.role == "GK":
                self.def_zone = (0, 10, 30, 50)
                self.mid_zone = (0, 10, 30, 50)
                self.att_zone = (0, 10, 30, 50)
            elif self.role in ["LCB", "RCB"]:
                self.def_zone = (10, 30, 20, 60)
                self.mid_zone = (20, 40, 20, 60)
                self.att_zone = (30, 50, 20, 60)
            elif self.role == "LB":
                self.def_zone = (10, 30, 0, 30)
                self.mid_zone = (20, 50, 0, 30)
                self.att_zone = (40, 80, 0, 30)
            elif self.role == "RB":
                self.def_zone = (10, 30, 50, 80)
                self.mid_zone = (20, 50, 50, 80)
                self.att_zone = (40, 80, 50, 80)
            elif self.role == "CDM":
                self.def_zone = (25, 45, 30, 50)
                self.mid_zone = (35, 55, 30, 50)
                self.att_zone = (45, 65, 30, 50)
            elif self.role == "LCM":
                self.def_zone = (30, 50, 20, 40)
                self.mid_zone = (40, 60, 20, 40)
                self.att_zone = (50, 80, 20, 40)
            elif self.role == "RCM":
                self.def_zone = (30, 50, 40, 60)
                self.mid_zone = (40, 60, 40, 60)
                self.att_zone = (50, 80, 40, 60)
            elif self.role == "LW":
                self.def_zone = (45, 65, 5, 25)
                self.mid_zone = (55, 75, 5, 25)
                self.att_zone = (75, 110, 5, 25)
            elif self.role == "RW":
                self.def_zone = (45, 65, 55, 75)
                self.mid_zone = (55, 75, 55, 75)
                self.att_zone = (75, 110, 55, 75)
            elif self.role == "ST":
                self.def_zone = (50, 70, 30, 50)
                self.mid_zone = (60, 80, 30, 50)
                self.att_zone = (80, 115, 30, 50)
        else:
            # Away team zones (just mirrored here)
            if self.role == "GK":
                self.def_zone = (110, 120, 30, 50)
                self.mid_zone = (110, 120, 30, 50)
                self.att_zone = (110, 120, 30, 50)
            elif self.role in ["LCB", "RCB"]:
                self.def_zone = (90, 110, 20, 60)
                self.mid_zone = (80, 100, 20, 60)
                self.att_zone = (70, 90, 20, 60)
            elif self.role == "LB":
                self.def_zone = (90, 110, 0, 30)
                self.mid_zone = (70, 100, 0, 30)
                self.att_zone = (40, 80, 0, 30)
            elif self.role == "RB":
                self.def_zone = (90, 110, 50, 80)
                self.mid_zone = (70, 100, 50, 80)
                self.att_zone = (40, 80, 50, 80)
            elif self.role == "CDM":
                self.def_zone = (75, 95, 30, 50)
                self.mid_zone = (65, 85, 30, 50)
                self.att_zone = (55, 75, 30, 50)
            elif self.role == "LCM":
                self.def_zone = (70, 90, 20, 40)
                self.mid_zone = (60, 80, 20, 40)
                self.att_zone = (40, 70, 20, 40)
            elif self.role == "RCM":
                self.def_zone = (70, 90, 40, 60)
                self.mid_zone = (60, 80, 40, 60)
                self.att_zone = (40, 70, 40, 60)
            elif self.role == "LW":
                self.def_zone = (55, 75, 5, 25)
                self.mid_zone = (45, 65, 5, 25)
                self.att_zone = (10, 45, 5, 25)
            elif self.role == "RW":
                self.def_zone = (55, 75, 55, 75)
                self.mid_zone = (45, 65, 55, 75)
                self.att_zone = (10, 45, 55, 75)
            elif self.role == "ST":
                self.def_zone = (50, 70, 30, 50)
                self.mid_zone = (40, 60, 30, 50)
                self.att_zone = (5, 40, 30, 50)



  def make_pass(self, game_state):
        #Find eligible targets
        eligible_targets = []
        for role, player in game_state.players[self.team].items():
            if player != self:  # Don't pass to self
                distance = np.sqrt((player.x_pos - self.x_pos)**2 + (player.y_pos - self.y_pos)**2)
                if distance < 40 and distance > 0.01: #Pass range, avoid zero distance
                    eligible_targets.append((player, distance))

        if eligible_targets:
            #Weighting players by distance
            total_weights = sum([1/distance for player, distance in eligible_targets])
            probabilities = [(1/distance)/total_weights for player, distance in eligible_targets]

            #Choose target
            chosen_target = random.choices([player for player, distance in eligible_targets], probabilities)[0]

            #Successful Pass
            success = random.random() < (self.attributes['passing_skill'] / 10)
            game_state.update_pass_data(self.x_pos, self.y_pos, chosen_target.x_pos, chosen_target.y_pos, success)
            game_state.ball_x = chosen_target.x_pos
            game_state.ball_y = chosen_target.y_pos

            #Change possession on failed pass
            if not success:
                game_state.lose_possession()

            print(f"{self.name} passed to {chosen_target.name}. Success: {success}")
        else:
            #No available targets, retain possession but log
            print (f"{self.name} has no available pass targets")



  def enhanced_pos_adjust(self, ball_x, ball_y, ball_location, game_state):
    """Enhanced position adjustment based on game context"""

    # Determine base position based on ball location
    if ball_location == "def_third":
        base_zone = self.def_zone
    elif ball_location == "mid":
        base_zone = self.mid_zone
    else:
        base_zone = self.att_zone

    # Get base position (center of appropriate zone)
    base_x = (base_zone[0] + base_zone[1]) / 2
    base_y = (base_zone[2] + base_zone[3]) / 2

    target_x = base_x
    target_y = base_y

    # Adjust for tactical considerations
    team_in_possession = game_state.possession
    team_style = game_state.team_style[self.team]
    score_diff = game_state.scoreline["home"] - game_state.scoreline["away"]
    time_remaining = 90 - game_state.time

    if self.team == "home":
        if score_diff < 0 and time_remaining < 15:
            # Push more players forward when losing late in the game
            base_x += 10
        else:
            if score_diff > 0 and time_remaining < 15:
                # Push more players forward when losing late in the game
                base_x -= 10

    if team_in_possession == self.team:
        # Team has the ball - make attacking movements
        if self.role in ["LW", "RW", "ST"]:
            # Attackers make runs into space
            self.make_attacking_run(ball_x, ball_y, game_state)
            return  # Skip standard movement
        elif self.role in ["LCM", "RCM", "CDM"]:
            # Midfielders offer support
            self.offer_passing_support(ball_x, ball_y)
            return  # Skip standard movement
        else:
            # All other players (e.g., defenders, GK) *consider* a pass
            # Check if player is close enough to ball for passing
            distance_to_ball = np.sqrt((self.x_pos - ball_x)**2 + (self.y_pos - ball_y)**2)
            if distance_to_ball < 15:  # Adjust threshold as needed
                if random.random() < 0.7: # 70% chance to make a pass
                    self.make_pass(game_state)
                    return # Skip standard movement
            else:
                # If not close enough, consider finding open space
                if random.random() < 0.4:  # 40% chance to find open space
                    best_space = self.find_open_space(game_state)
                    if best_space:
                        self.move_to(best_space[0], best_space[1], speed=self.attributes['speed'] * (self.energy / 100))
                        return

    else:
        # Defensive positioning when not in possession
        if team_style == "counter attack":
            # Counter teams maintain some shape
            defensive_distance = 25
        else:
            # Regular teams move further back
            defensive_distance = 35

        if self.role in ["LCB", "RCB"]:
            # Maintain spacing between centerbacks
            partner_role = "RCB" if self.role == "LCB" else "LCB"
            partner = game_state.players[self.team][partner_role]
            partner_x, partner_y = partner.get_position()

            # Basic avoidance: stay at least 10 units apart in x
            min_distance = 10
            if abs(self.x_pos - partner_x) < min_distance:
                if self.x_pos < partner_x:
                    target_x = max(self.def_zone[0], partner_x - min_distance)  # Move left
                else:
                    target_x = min(self.def_zone[1], partner_x + min_distance)  # Move right
            else:
                target_x = base_x  # If not close to partner, return to regular positioning

            if ball_location != "def_third":
              target_y = min(max(partner_y, self.def_zone[2]), self.def_zone[3])
            else:
            # Move towards the ball but stay within defensive zone
              target_y = min(max(ball_y, self.def_zone[2]), self.def_zone[3])  # Constrain y position

            self.move_to(target_x, target_y, speed=self.attributes['speed'] * (self.energy / 100))
            return  # Early return to override regular movement

        elif self.role in ["LB", "RB"]:
            # Move towards the ball
            target_x = ball_x - defensive_distance  # Move in front of ball
            target_y = ball_y
            self.move_to(target_x, target_y, speed=self.attributes['speed'] * (self.energy / 100))
            return

        elif self.role in ["LCM", "RCM", "CDM"]:
            # Move towards the ball
            target_x = ball_x - defensive_distance
            target_y = ball_y
            self.move_to(target_x, target_y, speed=self.attributes['speed'] * (self.energy / 100))
            return

    # Adjust the player to the target destination
    self.move_to(target_x, target_y, speed=self.attributes['speed'] * (self.energy / 100))



  def press_opponent(self, game_state):  #brother GPT
        if game_state.possession != self.team:
            nearest_opponent = game_state.get_nearest_player(game_state.possession, self.x_pos, self.y_pos)
            opp_x, opp_y = game_state.players[game_state.possession][nearest_opponent].get_position()

            # Move towards the opponent with the ball
            press_x = self.x_pos + (opp_x - self.x_pos) * 0.7
            press_y = self.y_pos + (opp_y - self.y_pos) * 0.7

            self.move_to(press_x, press_y, speed=self.attributes['speed'] * 1.1)

  def find_open_space(self, game_state):  #brother GPT
        teammates = game_state.players[self.team]
        opponents = game_state.players["away" if self.team == "home" else "home"]

        best_space = None
        max_space = 0

        for x in range(0, 120, 5):
            for y in range(0, 80, 5):
                space = self.calculate_space(x, y, teammates, opponents)
                if space > max_space:
                    max_space = space
                    best_space = (x, y)

        return best_space

  def calculate_space(self, x, y, teammates, opponents): #brother GPT
        space = 0
        for player in teammates.values():
            if player != self:
                space -= 1 / (((player.x_pos - x)**2 + (player.y_pos - y)**2)**0.5 + 1)
        for player in opponents.values():
            space -= 2 / (((player.x_pos - x)**2 + (player.y_pos - y)**2)**0.5 + 1)
        return space





  def make_attacking_run(self, ball_x, ball_y, game_state):
        """Make intelligent attacking runs based on game situation"""
        team = self.team

        # If we're in attacking third, make runs into the box
        if (team == "home" and ball_x > 80) or (team == "away" and ball_x < 40):
            if self.role == "ST":
                # Striker runs to far post or cuts back based on ball position
                if team == "home":
                    target_x = min(ball_x + 10, 115)
                    target_y = 40 + (20 if ball_y < 40 else -20)
                else:
                    target_x = max(ball_x - 10, 5)
                    target_y = 40 + (20 if ball_y < 40 else -20)
            elif self.role in ["LW", "RW"]:
              run_type = random.choice(["cut_inside", "overlap", "diagonal"])
              if run_type == "cut_inside":
                target_x = ball_x + (15 if self.team == "home" else -15)
                target_y = 40  # Move towards center
              elif run_type == "overlap":
                target_x = ball_x + (20 if self.team == "home" else -20)
                target_y = self.y_pos  # Maintain wide position
              else:  # diagonal
                target_x = ball_x + (15 if self.team == "home" else -15)
                target_y = 40 + (10 if self.y_pos < 40 else -10)

            # Execute the run with appropriate speed
            move_speed = self.attributes['speed'] * (self.energy / 100) * 1.2  # Sprinting
            self.move_to(target_x, target_y, speed=move_speed)

            # Sprinting reduces energy faster
            self.energy = max(self.energy - 0.3, 30)

  def offer_passing_support(self, ball_x, ball_y):
        """Offer supporting angles for the player in possession"""
        angle = random.randint(0, 360) * (math.pi / 180)  # Random angle in radians
        distance = random.randint(10, 20)  # Support distance

        # Calculate support position
        target_x = ball_x + distance * math.cos(angle)
        target_y = ball_y + distance * math.sin(angle)

        # Ensure position is within field boundaries
        target_x = max(min(target_x, 118), 2)
        target_y = max(min(target_y, 78), 2)

        # Move to support position
        move_speed = self.attributes['acceleration'] * (self.energy / 100)
        self.move_to(target_x, target_y, speed=move_speed)

        # Reduce energy slightly
        self.energy = max(self.energy - 0.15, 30)


  def position_change(self, game_state):
      current_time = game_state.time
      if current_time - self.last_movement >= self.move_cooldown:
        return

      ball_x, ball_y = game_state.ball_x, game_state.ball_y
      current_team = game_state.possession

      if self.dist_to_ball(game_state) < 20:
        self.move_to_default_pos(game_state)
      else:
        if current_team == self.team:
          self.off_movement(game_state)
        else:
          self.def_movement(game_state)

      self.last_movement = current_time




  def off_movement(self, game_state):
    open_space = self.find_open_space(game_state)
    if open_space:
      self.move_to(open_space[0], open_space[1])
    else:
      self.move_to_default_pos

  def def_movement(self, game_state):
    if self.should_press(game_state):
      self.press_opponent(game_state)
    else:
      self.move_to_defend_pos()


  def should_press(self, game_state):
    ball_dist = self.dist_to_ball(game_state)
    return ball_dist < 15 and random.random() <self.attributes['aggression']  / 10 #change attribute later

  def move_to_default_pos(self, game_state):
    original_pos = game_state.original_positions[self.team][self.role]
    self.move_to(original_pos[0], original_pos[1])

  def move_to_defend_pos(self, game_state):
    ball_x, ball_y = game_state.ball_x, game_state.ball_y
    target_x = self.x_pos + (ball_x - self.x_pos) * 0.3
    target_y = self.y_pos + (ball_y - self.y_pos) * 0.3
    self.move_to(target_x, target_y)

  def dist_to_ball(self, game_state):
    return ((self.x_pos - game_state.ball_x)**2 + (self.y_pos - game_state.ball_y)**2)**0.5


  #brother perplexity:
  def return_to_formation(self, game_state):
        current_phase = self.determine_game_phase(game_state)
        target_pos = self.base_position[current_phase]
        self.move_to(target_pos[0], target_pos[1], speed=self.attributes['speed'] * 0.5)

  def make_intelligent_run(self, game_state):
        if self.role in ['ST', 'LW', 'RW']:
            # Implement logic for attacking runs
            if game_state.possession == self.team:
                space_behind = self.find_space_behind_defense(game_state)
                if space_behind:
                    self.move_to(space_behind[0], space_behind[1], speed=self.attributes['speed'] * 1.2)
        elif self.role in ['LCM', 'RCM', 'CAM']:
            # Implement logic for midfield support runs
            if game_state.possession == self.team:
                support_position = self.find_support_position(game_state)
                self.move_to(support_position[0], support_position[1])

  def find_space_behind_defense(self, game_state):
        # Simplified logic to find space behind the defense
        if self.team == 'home':
            return (min(self.x_pos + 15, 118), self.y_pos + random.uniform(-10, 10))
        else:
            return (max(self.x_pos - 15, 2), self.y_pos + random.uniform(-10, 10))

  def find_support_position(self, game_state):
        # Simplified logic to find a support position
        ball_carrier = game_state.get_nearest_player(self.team, game_state.ball_x, game_state.ball_y)
        x_offset = 10 if game_state.ball_x < 60 else -10
        y_offset = random.uniform(-5, 5)
        return (ball_carrier.x_pos + x_offset, ball_carrier.y_pos + y_offset)

  def aggressive_pass(self, game_state):
        # Implement risk-reward system for aggressive passes
        potential_targets = self.find_forward_pass_targets(game_state)
        if potential_targets:
            target = max(potential_targets, key=lambda x: x[1])  # Choose the highest-rated target
            success_chance = self.calculate_pass_success(target[0], game_state)
            if random.random() < success_chance:
                return target[0]
        return None

  def find_forward_pass_targets(self, game_state):
        targets = []
        for role, player in game_state.players[self.team].items():
            if player != self and player.x_pos > self.x_pos:
                distance = ((player.x_pos - self.x_pos)**2 + (player.y_pos - self.y_pos)**2)**0.5
                rating = (player.x_pos - self.x_pos) / distance  # Prioritize forward passes
                targets.append((player, rating))
        return targets

  def calculate_pass_success(self, target, game_state):
        distance = ((target.x_pos - self.x_pos)**2 + (target.y_pos - self.y_pos)**2)**0.5
        base_success = max(0.1, 1 - (distance / 100))  # Longer passes are riskier
        skill_factor = self.attributes['passing_skill'] / 10
        return min(0.95, base_success * skill_factor)  # Cap at 95% success rate

  def make_decision(self, game_state):
        if game_state.possession == self.team:
            # Offensive decision making
            if self.should_shoot(game_state):
                return 'shoot'
            elif self.should_dribble(game_state):
                return 'dribble'
            else:
                aggressive_pass_target = self.aggressive_pass(game_state)
                if aggressive_pass_target:
                    return 'aggressive_pass', aggressive_pass_target
                else:
                    return 'safe_pass'
        else:
            # Defensive decision making
            if self.should_tackle(game_state):
                return 'tackle'
            else:
                return 'position'

  def should_shoot(self, game_state):
        # Simplified shooting decision
        if self.role in ['ST', 'LW', 'RW']:
            distance_to_goal = ((self.x_pos - 120)**2 + (self.y_pos - 40)**2)**0.5
            return distance_to_goal < 25 and random.random() < self.attributes['shooting'] / 10

  def should_dribble(self, game_state):
        # Simplified dribbling decision
        space_ahead = self.find_open_space(game_state)
        if space_ahead:
            distance_to_space = ((space_ahead[0] - self.x_pos)**2 + (space_ahead[1] - self.y_pos)**2)**0.5
            return distance_to_space < 10 and random.random() < self.attributes['dribbling'] / 10

  def should_tackle(self, game_state):
        # Simplified tackling decision
        ball_carrier = game_state.get_nearest_player(game_state.possession, game_state.ball_x, game_state.ball_y)
        distance_to_ball = ((ball_carrier.x_pos - self.x_pos)**2 + (ball_carrier.y_pos - self.y_pos)**2)**0.5
        return distance_to_ball < 5 and random.random() < self.attributes['tackling'] / 10


## Bayesian Player (?)

In [526]:

class BayesianPitchLearning: # placeholder class
  def __init__(self, pitch_width=120, pitch_height=80, cell_size=5):
    pass
  def update_belief(self, x_pos, y_pos, role, action_type, success):
    pass
  def suggest_movement(self, x_pos, y_pos, role, radius=20, team="home", phase="attack"):
    return x_pos+random.uniform(-5, 5), y_pos+random.uniform(-5, 5)
  def get_effectiveness_probability(self, x, y, role, action_type):
    return random.uniform(0.1, 0.9)


In [527]:
# Global Bayesian learning system
pitch_learning = BayesianPitchLearning(pitch_width=120, pitch_height=80, cell_size=5)

class PlayerMechanicsBayesian(PlayerMechanics):
    def __init__(self, name, x_pos, y_pos, role, team, attributes=None):
        super().__init__(name, x_pos, y_pos, role, team, attributes)

        # Confidence in Bayesian model (increases as match progresses)
        self.model_confidence = 0.2

        # Track action outcomes for learning
        self.recent_actions = []
        self.max_action_memory = 20

        # Learning rate - how quickly player adapts to new information
        self.learning_rate = self.attributes.get('vision', 7) / 10

        # Areas where player has been successful (tracked locally)
        self.successful_positions = []

    def record_action(self, action_type, success, x_pos=None, y_pos=None):
        """Record an action outcome for learning"""
        if x_pos is None:
            x_pos = self.x_pos
        if y_pos is None:
            y_pos = self.y_pos

        # Add to recent actions queue
        self.recent_actions.append({
            'type': action_type,
            'success': success,
            'x': x_pos,
            'y': y_pos,
            'game_time': None  # Would be filled from actual game time
        })

        # Limit size of recent actions
        if len(self.recent_actions) > self.max_action_memory:
            self.recent_actions.pop(0)

        # If action was successful, remember this position
        if success:
            self.successful_positions.append((x_pos, y_pos))
            if len(self.successful_positions) > 10:
                self.successful_positions.pop(0)

        # Update the global Bayesian model
        pitch_learning.update_belief(x_pos, y_pos, self.role, action_type, success)

        # Increase confidence in model as more actions are recorded
        self.model_confidence = min(0.9, self.model_confidence + 0.01)

    def bayesian_position_adjust(self, ball_x, ball_y, game_state):
        """Use Bayesian model to adjust position intelligently"""
        # Get game phase from ball position and team
        if self.team == game_state.possession:
            if (self.team == "home" and ball_x > 80) or (self.team == "away" and ball_x < 40):
                phase = "attack"
            elif (self.team == "home" and ball_x < 40) or (self.team == "away" and ball_x > 80):
                phase = "transition"
            else:
                phase = "build_up"
        else:
            if (self.team == "home" and ball_x < 40) or (self.team == "away" and ball_x > 80):
                phase = "defense"
            else:
                phase = "defensive_transition"

        # Maximum movement distance varies by phase
        if phase in ["attack", "defensive_transition"]:
            max_distance = 25
        elif phase in ["defense", "build_up"]:
            max_distance = 15
        else:  # transition
            max_distance = 20

        # Factor in player's energy level
        max_distance *= (self.energy / 100)

        # Factor in urgency based on score and time
        if game_state.time > 75:  # Late game
            if ((self.team == "home" and game_state.score["home"] < game_state.score["away"]) or
                (self.team == "away" and game_state.score["away"] < game_state.score["home"])):
                # Losing late - increase urgency
                max_distance *= 1.2

        # Get suggested movement from Bayesian model
        target_x, target_y = pitch_learning.suggest_movement(
            self.x_pos, self.y_pos, self.role,
            radius=max_distance, team=self.team, phase=phase
        )

        # Blend with basic positioning based on confidence in model
        if self.model_confidence < 0.5:
            # If confidence is low, rely more on default positioning
            if phase == "attack":
                base_zone = self.att_zone
            elif phase == "defense":
                base_zone = self.def_zone
            else:
                base_zone = self.mid_zone

            default_x = (base_zone[0] + base_zone[1]) / 2
            default_y = (base_zone[2] + base_zone[3]) / 2

            # Blend default with Bayesian suggestion
            blend_factor = self.model_confidence * 2  # 0-1 range
            target_x = (1 - blend_factor) * default_x + blend_factor * target_x
            target_y = (1 - blend_factor) * default_y + blend_factor * target_y

        # Execute movement with appropriate speed
        move_speed = self.attributes['speed'] * (self.energy / 100)
        self.move_to(target_x, target_y, speed=move_speed)

        # Record the position in history
        self.position_history.append((self.x_pos, self.y_pos))
        if len(self.position_history) > 50:
            self.position_history.pop(0)

        # Update energy
        self.energy = max(self.energy - 0.1, 30)

        return target_x, target_y

    def enhanced_pos_adjust(self, ball_x, ball_y, ball_location, game_state):
        """Override with Bayesian learning version"""
        # First use Bayesian positioning if we have enough data
        if random.random() < self.model_confidence:
            target_x, target_y = self.bayesian_position_adjust(ball_x, ball_y, game_state)
            return  # Use Bayesian positioning

        # Fall back to original enhanced_pos_adjust if bayesian positioning not used
        super().enhanced_pos_adjust(ball_x, ball_y, ball_location, game_state)

    def assess_action_probability(self, action_type, x, y, game_state):
        """
        Calculate probability of success for a specific action at position

        Args:
            action_type: Type of action (pass, shot, etc.)
            x, y: Position coordinates
            game_state: Current game state

        Returns:
            float: Probability of success (0-1)
        """
        # Get base probability from Bayesian model
        base_prob = pitch_learning.get_effectiveness_probability(x, y, self.role, action_type)

        # Adjust for player attributes
        attribute_factor = 1.0
        if action_type == "pass":
            attribute_factor = self.attributes.get('passing_skill', 7) / 7
        elif action_type == "shot":
            attribute_factor = self.attributes.get('shooting', 7) / 7
        elif action_type == "dribble":
            attribute_factor = self.attributes.get('agility', 7) / 7
        elif action_type in ["tackle", "interception"]:
            attribute_factor = self.attributes.get('tackling', 7) / 7

        # Adjust for energy levels
        energy_factor = 0.5 + 0.5 * (self.energy / 100)

        # Calculate final probability
        final_prob = base_prob * attribute_factor * energy_factor

        # Clip to range [0.1, 0.95] to avoid extremes
        return max(0.1, min(0.95, final_prob))

    def choose_action(self, game_state, available_actions):
        """
        Choose best action based on Bayesian probabilities

        Args:
            game_state: Current game state
            available_actions: List of available action types

        Returns:
            str: Chosen action type
        """
        best_action = None
        best_value = -1

        for action in available_actions:
            # Calculate success probability
            prob = self.assess_action_probability(action, self.x_pos, self.y_pos, game_state)

            # Calculate expected value (could factor in game state)
            value = prob

            # Add strategic considerations
            if action == "shot" and game_state.time > 85 and game_state.score[self.team] < game_state.score["away" if self.team == "home" else "home"]:
                # More likely to shoot when losing late
                value *= 1.5

            if value > best_value:
                best_value = value
                best_action = action

        return best_action

    def make_attacking_run(self, ball_x, ball_y, game_state):
        """Make intelligent attacking runs based on game situation and Bayesian model"""
        team = self.team

        # Check if we have historical successful positions in attacking areas
        attack_positions = []
        for pos_x, pos_y in self.successful_positions:
            # Filter positions in attacking third
            if (team == "home" and pos_x > 80) or (team == "away" and pos_x < 40):
                attack_positions.append((pos_x, pos_y))

        # If we have successful attacking positions, prefer those
        if attack_positions and random.random() < self.model_confidence:
            # Choose a random successful position with bias toward more recent ones
            weights = [i+1 for i in range(len(attack_positions))]
            chosen_pos = random.choices(attack_positions, weights=weights, k=1)[0]

            # Add some randomness to exact target
            target_x = chosen_pos[0] + random.uniform(-5, 5)
            target_y = chosen_pos[1] + random.uniform(-5, 5)

            # Execute the run
            move_speed = self.attributes['speed'] * (self.energy / 100) * 1.2  # Sprinting
            self.move_to(target_x, target_y, speed=move_speed)

            # Sprinting reduces energy faster
            self.energy = max(self.energy - 0.3, 30)
            return

        # If no successful positions or not using them, use Bayesian model
        if random.random() < self.model_confidence:
            phase = "attack"
            max_distance = 30  # Longer runs for attacking players

            # Get suggested position from model
            target_x, target_y = pitch_learning.suggest_movement(
                self.x_pos, self.y_pos, self.role,
                radius=max_distance, team=self.team, phase=phase
            )

            # Execute the run with appropriate speed
            move_speed = self.attributes['speed'] * (self.energy / 100) * 1.2  # Sprinting
            self.move_to(target_x, target_y, speed=move_speed)

            # Sprinting reduces energy faster
            self.energy = max(self.energy - 0.3, 30)
            return

        # Fall back to original method if not using Bayesian
        super().make_attacking_run(ball_x, ball_y, game_state)

# Pass Class

In [528]:
class Pass:
  def __init__(self, game_state):
    self.game_state = game_state
    self.pass_data = []
    self.pass_history = []
    self.data_file = "pass_history.csv"
    self.load_pass_history()


  #-----------------CSV Updating------------------------------------

  def load_pass_history(self):                #SAVE PLAYER POS FOR EACH PASS FOR DEBUGGING
    try:
       with open(self.data_file, mode='r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    self.pass_history.append({
                        'pass_type': row['pass_type'],
                        'distance': float(row['distance']),
                        'pressure': float(row['pressure']),
                        'outcome': row['outcome'],
                        'player_positions': float(row['player_positions']),
                        'start': (float(row['passer_x']), float(row['passer_y'])),
                        'end' : (float(row['target_x']), float(row['target_y']))
                    })
    except FileNotFoundError:
        print("No historical pass data found. Creating a new dataset.")

  def save_pass_history(self):
        """Saves passing history to CSV file for future Bayesian updates."""
        with open(self.data_file, mode='w', newline='') as file:
            fieldnames = ['pass_type', 'distance', 'pressure', 'outcome', 'player_positions', 'start', 'end']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(self.pass_history)


  def update_passing_history(self, pass_type, distance, pressure, success, passer_x, passer_y, target_x, target_y):
    player_positions = self.game_state.get_player_pos()
    self.pass_history.append({
      'pass_type': pass_type,
      'distance': distance,
      'pressure': pressure,
      'outcome': 'success' if success else 'failed',
      'player_positions' : player_positions,
      'start' : (passer_x, passer_y),
      'end': (target_x, target_y),
    })
    self.save_pass_history()


  #--------------------------------Calculating Passes----------------

  #Determining the Field Zone

  def get_field_zone(self, x, y):
    if x < 30:
        vertical_zone = "defensive"
    elif x < 80:
        vertical_zone = "middle"
    else:
        vertical_zone = "attacking"

    if y < 25:
        horizontal_zone = "left"
    elif y < 55:
        horizontal_zone = "central"
    else:
        horizontal_zone = "right"

    return f"{vertical_zone}_{horizontal_zone}"


  #Calculating the Targets Value (How valuable the pass will be) [BROTHER CLAUDE]

  def calculate_target_value(self, passer, target_player, target_x, target_y):
    team = self.game_state.possession
    passer_x, passer_y = passer.get_position()

    # Distance to goal
    if team == "home":
        goal_x, goal_y = 120, 40
        progression_to_goal = (passer_x - target_x) / 120  # Negative is good
    else:
        goal_x, goal_y = 0, 40
        progression_to_goal = (target_x - passer_x) / 120  # Positive is good

    distance_to_goal_before = np.sqrt((goal_x - passer_x)**2 + (goal_y - passer_y)**2)
    distance_to_goal_after = np.sqrt((goal_x - target_x)**2 + (goal_y - target_y)**2)

    # Goal proximity improvement
    goal_proximity_improvement = (distance_to_goal_before - distance_to_goal_after) / 120

    # Space value
    pressure = self.calc_def_pressure(target_x, target_y)
    space_value = 1 - pressure

    # Pass difficulty
    distance = np.sqrt((target_x - passer_x)**2 + (target_y - passer_y)**2)
    pass_type = self.pass_type(distance)
    pass_difficulty = 1 - self.calc_pass_prob(passer, target_x, target_y, pressure, pass_type)

    # Central position bonus (central positions are more valuable)
    central_bonus = 0
    if 30 < target_y < 50:  # Middle third vertically
        central_bonus = 0.1

    # Attacking third bonus
    attacking_third_bonus = 0
    if (team == "home" and target_x > 80) or (team == "away" and target_x < 40):
        attacking_third_bonus = 0.15

    # Player attribute bonuses
    target_player_attributes = getattr(target_player, 'attributes', {})

    # Different weightings based on field zone
    zone = self.get_field_zone(passer_x, passer_y)

    if "attacking" in zone:
        # In attacking third, prioritize shooting and key pass attributes
        offensive_skill = (target_player_attributes.get('finishing', 5) +
                          target_player_attributes.get('attacking_position', 5)) / 20
        value = (goal_proximity_improvement * 0.4 +
                offensive_skill * 0.3 +
                space_value * 0.2 +
                central_bonus * 0.1 -
                pass_difficulty * 0.2)

    elif "middle" in zone:
        # In middle third, balance progression and possession
        playmaking_skill = (target_player_attributes.get('passing', 5) +
                           target_player_attributes.get('vision', 5)) / 20
        value = (goal_proximity_improvement * 0.3 +
                playmaking_skill * 0.2 +
                space_value * 0.3 +
                central_bonus * 0.1 -
                pass_difficulty * 0.1)

    else:
        # In defensive third, prioritize safety and building up play
        build_up_skill = (target_player_attributes.get('composure', 5) +
                         target_player_attributes.get('passing', 5)) / 20
        value = (space_value * 0.4 +
                build_up_skill * 0.3 +
                goal_proximity_improvement * 0.2 -
                pass_difficulty * 0.3)

    # Add attacking bonus for all zones
    value += attacking_third_bonus

    return value


  def calc_pass_prob(self, passer, target_x, target_y, pressure, pass_type):
    passing_skill = getattr(passer, 'attributes', {}).get('passing',7)
    passer_x, passer_y = passer.get_position()
    distance = np.sqrt((target_x - passer_x)**2 + (target_y - passer_y)**2)
    base_probabilites = {'short': 0.85, 'medium': 0.7, 'long': 0.5 }
    start_prob = base_probabilites.get(pass_type, 0.7)

    skill_mod = (passing_skill - 5) / 10
    prob_with_skill_mod = base_probabilites.get(pass_type, 0.7) * (1 + skill_mod)
    pressure_mod = 1 - (pressure * 0.5)
    adjusted_prob = prob_with_skill_mod * pressure_mod

    matching_passes = [
            p for p in self.pass_history
            if p['pass_type'] == pass_type
            and abs(p['distance'] - distance) < 5
            and abs(p['pressure'] - pressure) < 0.1
        ]

    successful_passes = sum(1 for p in matching_passes if p['outcome'] == 'success')
    total_passes = max(len(matching_passes),1)
    bayesian_prob = (successful_passes + 1) / (total_passes + 2)

    final_prob = (adjusted_prob * 0.7) + (bayesian_prob * 0.3)
    return min(0.99, max(0.05, final_prob))


  def det_pass_target(self, passer):
    team = self.game_state.possession
    options = []

    for role, player in self.game_state.players[team].items():
        if player == passer:
            continue

        player_x, player_y = player.get_position()
        pass_value = self.calculate_target_value(passer, player, player_x, player_y)

        # Calculate distance for filtering
        passer_x, passer_y = passer.get_position()
        distance = np.sqrt((player_x - passer_x)**2 + (player_y - passer_y)**2)

        # Filter out unrealistic passes
        if distance > 70:  # Max realistic pass distance
            continue

        # Store as potential option
        options.append({
            'player': player,
            'position': (player_x, player_y),
            'distance': distance,
            'value': pass_value
        })

    # Sort by value
    if options:
        # Add slight randomness to prevent predictable passing
        for option in options:
            option['value'] += random.uniform(-0.05, 0.05)

        options.sort(key=lambda x: x['value'], reverse=True)
        best = options[0]
        return (best['player'], best['position'][0], best['position'][1], best['distance'])

    return None


  def pass_execution(self):
    current_team = self.game_state.possession
    passer_role = self.game_state.get_nearest_player(current_team, self.game_state.ball_x, self.game_state.ball_y)
    current_passer = self.game_state.players[current_team][passer_role]
    passer_x, passer_y = current_passer.get_position()

    # Get passing options based on player positions and passing attributes
    if (current_team == "home" and passer_x > 80) or (current_team == "away" and passer_x < 40):
        passing_options = self.final_3rd_strat(current_passer)
    else:
        passing_options = self.det_pass_target(current_passer)


    if not passing_options:
        print("No passing options available!")
        self.game_state.lose_possession()
        return False

    # Calculate success probability based on pass difficulty and player attributes
    receiver, target_x, target_y, distance = passing_options
    pressure = self.calc_def_pressure(target_x, target_y)
    pass_type = self.pass_type(distance)

    success_probability = self.calc_pass_prob(current_passer, target_x, target_y, pressure, pass_type) # Assign success_probability a value
    success_probability = self.risk_by_state(success_probability)

    pass_success = random.random() < success_probability
    self.update_passing_history(self, pass_type, distance, pressure, success, passer_x, passer_y, target_x, target_y )


    if pass_success:
            current_passer.record_action("pass", True, target_x, target_y)  # Record successful pass for passer
            receiver.record_action("receive", True, target_x, target_y)  # Record successful receive for receiver

            receiver.move_to(target_x, target_y, speed=7)
            self.game_state.ball_x, self.game_state.ball_y = receiver.get_position()
            print(f"✅ Successful {pass_type} pass: {passer_role} → {receiver.name}")
            self.game_state.update_ball_location()
            self.pass_data.append([(passer_x, passer_y), (target_x, target_y)]) #add data
            return True
    else:
            current_passer.record_action("pass", False, target_x, target_y)  # Record failed pass for passer

            self.game_state.lose_possession()
            print(f"❌ {pass_type} pass failed: {passer_role} lost the ball")
            return False


  def pass_type(self, distance):
    if distance < 20:
            return "short"
    elif distance < 40:
            return "medium"
    elif distance < 60:
            return random.choice(["long"])

  def calc_def_pressure(self, x, y):
    oppo_team = "away" if self.game_state.possession == "home" else "home"
    pressure = 0

    for role, player in self.game_state.players[oppo_team].items():
      player_x, player_y = player.get_position()
      distance = np.sqrt((player_x - x)**2 + (player_y - y)**2)
      if distance < 10:
        pressure += (10 - distance) / 10

    return min(1.0, pressure)

  def risk_by_state(self, base_prob):
    team = self.game_state.possession
    score_differential = self.game_state.scoreline[team] - self.game_state.scoreline["away" if team == "home" else "home"]
    match_time = self.game_state.time

    if match_time > 80: #Last 10 minutes
      if score_differential < 0: #Losing
        return base_prob * 1.3
      elif score_differential > 0:  #Winning
        return base_prob * 0.85

    return base_prob



  def final_3rd_strat(self, passer):
    team = self.game_state.possession
    passer_x, passer_y = passer.get_position()
    final_third = (team == "home" and passer_x > 80) or (team == "away" and passer_x < 40)

    if not final_third:
      return self.det_pass_target(passer)

    options = []

    for role, player in self.game_state.players[team].items():
      if player == passer:
        continue

      player_x, player_y = player.get_position()
      distance = np.sqrt((player_x - passer_x**2) + (player_y - passer_y)**2)

      if distance > 35:
        continue

      shooting_pos = self.evaluate_shoot_pos(player_x, player_y, team)

      key_pass_val = shooting_pos * 0.6

      pressure = self.calc_def_pressure(player_x, player_y)
      space_val = (1 - pressure) * 0.3

      pass_val = key_pass_val + space_val

      options.append({
        'player': player,
        'position': (player_x, player_y),
        'distance': distance,
        'value': pass_val
      })

      if options:
        # Add slight randomness
        for option in options:
            option['value'] += random.uniform(-0.05, 0.05)

        options.sort(key=lambda x: x['value'], reverse=True)
        best = options[0]
        return (best['player'], best['position'][0], best['position'][1], best['distance'])

    return self.det_pass_target(passer)


  def evaluate_shoot_pos(self,x,y,team):

    if team == "home":
      goal_x, goal_y = 120,40
    else:
      goal_x, goal_y = 0,40


    distance_to_goal = np.sqrt((goal_x - x)**2 + (goal_y - y)**2)


    if team == "home":
      perp_dist = 120 - x
    else:
      perp_dist = x


    angle_fact = perp_dist / max(1, np.sqrt((y-goal_y)**2 + perp_dist**2))

    pressure = self.calc_def_pressure(x,y)

    dist_factor = max(0,1 - (distance_to_goal / 30))

    position_value = (dist_factor * 0.4) + (angle_fact * 0.3) + ((1-pressure) * 0.2)

    return position_value


# Shooting

In [529]:
class Shot:
  def __init__(self, game_state):
    self.game_state = game_state
    self.shot_history = []
    self.data_file = "shot_history.csv"
    self.load_shot_history()

  def load_shot_history(self):
    try:
       with open(self.data_file, mode='r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    self.shot_history.append({
                        'distance': float(row['distance']),
                        'angle': float(row['angle']),
                        'pressure': float(row['pressure']),
                        'outcome': row['outcome'],
                        'start': (float(row['shooter_x']), float(row['shooter_y'])),
                        'time' : float(row['time']),
                        'player_positions': float(row['player_positions']),
                        'event_type' : row['event_type']

                    })
    except FileNotFoundError:
      print("No historical data, making a new dataset")


  def save_shot_history(self):
    with open(self.data_file, mode='w', newline='') as file:
            fieldnames = ['event_type', 'player_positions', 'time', 'start', 'distance', 'angle', 'pressure', 'outcome']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(self.shot_history)

  def update_shot_history(self, distance, angle, pressure, success, shooter_x, shooter_y):
        outcome = 'goal' if success else 'miss'
        player_positions = self.game_state.get_player_pos()
        self.shot_history.append({
            'event_type':'shot',
            'player_positions': player_positions,
            'time': self.game_state.time,
            'start':(shooter_x, shooter_y),
            'distance': distance,
            'angle': angle,
            'pressure': pressure,
            'outcome': outcome
        })
        self.save_shot_history()

  def prob_modifier(self, base_prob, strength_fact):
        adjustment = (strength_fact - 5) * 0.05  # Less aggressive adjustment
        return min(max(base_prob * (1 + adjustment), 0.05), 0.85)

  def calc_shot_prob(self, shooter, target_x, target_y, pressure):
    shooter_skill = getattr(shooter, 'attributes', {}).get('shooting', 7)
    shooter_x, shooter_y = shooter.get_position()

    distance = np.sqrt((target_x - shooter_x)**2 + (target_y - shooter_y)**2)
    angle = abs(math.degrees(math.atan2(target_y - shooter_y, target_x - shooter_x)))
    base_prob = max(0.01, min(0.7, 1 - (distance / 100))) * (1 - (angle / 180))
    skill_mod = (shooter_skill - 5) / 10
    pressure_mod = 1 - (pressure * 0.5)
    adjusted_prob = base_prob * (1 + skill_mod) * pressure_mod

    matching_shots = [s for s in self.shot_history if abs(s['distance'] - distance) < 5 and abs(s['angle'] - angle) < 10 and abs(s['pressure'] - pressure) < 0.1]
    successful_shots = sum(1 for s in matching_shots if s['outcome'] == 'goal')
    total_shots = max(len(matching_shots), 1)
    bayesian_shot_prob = (successful_shots + 1) / (total_shots + 2)

    final_shot_prob = (adjusted_prob * 0.7) + (bayesian_shot_prob * 0.3)
    return min(0.95, max(0.01, final_shot_prob))


  def execute_shot(self):
        team = self.game_state.possession
        shooter_role = self.game_state.get_nearest_player(team, self.game_state.ball_x,self.game_state.ball_y)
        shooter = self.game_state.players[team][shooter_role]

        goal_x, goal_y = (120, 40) if team == "home" else (0, 40)
        pressure = self.calc_def_pressure(shooter.x_pos, shooter.y_pos)
        shot_probability = self.calc_shot_prob(shooter, goal_x, goal_y, pressure)
        shot_success = random.random() < shot_probability

        shooter_x = shooter.x_pos
        shooter_y = shooter.y_pos



        self.update_shot_history(shooter.x_pos, shooter.y_pos, pressure, shot_success, shooter_x, shooter_y)

        if shot_success:
            self.game_state.scoreline[team] += 1
            print(f"GOAL! {team.capitalize()} team scores!")
            print(f"New score: Home {self.game_state.scoreline['home']} - Away {self.game_state.scoreline['away']}")
            self.game_state.ball_x, self.game_state.ball_y = 60,40
            self.game_state.possession = "away" if team == "home" else "home" #use lose possession??
        else:
            if random.random() < 0.7:  # 70% chance goalkeeper saves
                print(f"Shot saved by the goalkeeper!")
                self.game_state.lose_possession()  # Goalkeeper has the ball
            else:
                print(f"Shot misses the target!")
                # Randomly decide if it's a goal kick or corner
                if random.random() < 0.5:
                    self.game_state.lose_possession()
                    if team == "home":
                        self.game_state.ball_x = 110  # Goal kick position for away team
                    else:
                        self.game_state.ball_x = 10   # Goal kick position for home team
                    self.game_state.ball_y = 40
                else:
                    # Corner kick - stay with same team
                    if team == "home":
                        self.game_state.ball_x = 115
                    else:
                        self.game_state.ball_x = 5
                    self.game_state.ball_y = random.choice([5, 75])  # Left or right corner

  def calc_def_pressure(self, x, y):
    def_team = "away" if self.game_state.possession == "home" else "home"
    pressure = 0
    for role, player in self.game_state.players[def_team].items():
            player_x, player_y = player.get_position()
            distance = np.sqrt((player_x - x)**2 + (player_y - y)**2)
            if distance < 10:
                pressure += (10 - distance) / 10
    return min(1.0, pressure)


# Advanced Shooting

In [530]:
from scipy.stats import beta

class BayesianShot(Shot):
    def __init__(self, game_state):
        super().__init__(game_state)
        # Parameters for Beta distribution priors
        self.distance_bins = np.linspace(0, 120, 13)  # 12 bins
        self.angle_bins = np.linspace(0, 90, 10)     # 9 bins
        self.pressure_bins = np.linspace(0, 1.0, 6)   # 5 bins

        # Initialize shot database with structured priors
        self.shot_db = self._initialize_shot_db()

        # If historical data exists, update the priors
        if self.shot_history:
            self._update_shot_db_from_history()

    def _initialize_shot_db(self):
        """Initialize the shot database with prior information"""
        shot_db = {}

        # Create bins for different conditions
        for d_idx in range(len(self.distance_bins) - 1):
            d_min, d_max = self.distance_bins[d_idx], self.distance_bins[d_idx + 1]
            for a_idx in range(len(self.angle_bins) - 1):
                a_min, a_max = self.angle_bins[a_idx], self.angle_bins[a_idx + 1]
                for p_idx in range(len(self.pressure_bins) - 1):
                    p_min, p_max = self.pressure_bins[p_idx], self.pressure_bins[p_idx + 1]

                    # Set prior based on football intuition
                    # Better chance when closer to goal and lower angle
                    distance_factor = 1 - ((d_min + d_max) / 2) / 120
                    angle_factor = 1 - ((a_min + a_max) / 2) / 90
                    pressure_factor = 1 - ((p_min + p_max) / 2)

                    # Prior probability (intuition-based)
                    prior_p = distance_factor * angle_factor * pressure_factor

                    # Beta distribution parameters (alpha=successes+1, beta=failures+1)
                    # Higher confidence in priors for common situations
                    confidence = 2  # Base confidence
                    if d_min < 40:  # More confidence in close shots
                        confidence += 2
                    if a_min < 30:  # More confidence in central shots
                        confidence += 2

                    # Create beta distribution parameters
                    alpha = max(1, prior_p * confidence)
                    beta_param = max(1, (1 - prior_p) * confidence)

                    key = (d_min, d_max, a_min, a_max, p_min, p_max)
                    shot_db[key] = {
                        'alpha': alpha,
                        'beta': beta_param,
                        'goals': 0,
                        'shots': 0
                    }

        return shot_db

    def _update_shot_db_from_history(self):
        """Update the shot database with historical data"""
        for shot in self.shot_history:
            distance = shot['distance']
            angle = shot['angle']
            pressure = shot['pressure']
            outcome = shot['outcome']

            # Find the correct bin for this shot
            bin_key = self._find_bin(distance, angle, pressure)
            if bin_key:
                self.shot_db[bin_key]['shots'] += 1
                if outcome == 'goal':
                    self.shot_db[bin_key]['goals'] += 1

    def _find_bin(self, distance, angle, pressure):
        """Find the bin corresponding to the shot parameters"""
        for d_idx in range(len(self.distance_bins) - 1):
            d_min, d_max = self.distance_bins[d_idx], self.distance_bins[d_idx + 1]
            if d_min <= distance < d_max:
                for a_idx in range(len(self.angle_bins) - 1):
                    a_min, a_max = self.angle_bins[a_idx], self.angle_bins[a_idx + 1]
                    if a_min <= angle < a_max:
                        for p_idx in range(len(self.pressure_bins) - 1):
                            p_min, p_max = self.pressure_bins[p_idx], self.pressure_bins[p_idx + 1]
                            if p_min <= pressure < p_max:
                                return (d_min, d_max, a_min, a_max, p_min, p_max)
        return None

    def _find_nearest_bins(self, distance, angle, pressure, num_bins=3):
        """Find the nearest bins for a given shot scenario"""
        distances = []
        for key in self.shot_db:
            d_min, d_max, a_min, a_max, p_min, p_max = key
            d_center = (d_min + d_max) / 2
            a_center = (a_min + a_max) / 2
            p_center = (p_min + p_max) / 2

            # Calculate Euclidean distance in normalized feature space
            dist = np.sqrt(((distance - d_center)/120)**2 +
                           ((angle - a_center)/90)**2 +
                           ((pressure - p_center)/1.0)**2)
            distances.append((key, dist))

        # Return the nearest bins
        nearest_bins = sorted(distances, key=lambda x: x[1])[:num_bins]
        return [key for key, _ in nearest_bins]

    def calc_bayesian_shot_prob(self, shooter, target_x, target_y, pressure):
        """Calculate shot probability using Bayesian model"""
        shooter_skill = getattr(shooter, 'attributes', {}).get('shooting', 7)
        shooter_x, shooter_y = shooter.get_position()

        # Calculate distance and angle
        distance = np.sqrt((target_x - shooter_x)**2 + (target_y - shooter_y)**2)
        raw_angle = math.degrees(math.atan2(target_y - shooter_y, target_x - shooter_x))
        angle = abs(raw_angle)  # Absolute angle from horizontal

        # Find the bin for this shot
        bin_key = self._find_bin(distance, angle, pressure)

        # If no exact bin found, use nearest bins
        if not bin_key:
            nearest_bins = self._find_nearest_bins(distance, angle, pressure)

            # Calculate weighted average probability from nearest bins
            if nearest_bins:
                probs = []
                weights = []

                for key in nearest_bins:
                    alpha = self.shot_db[key]['alpha'] + self.shot_db[key]['goals']
                    beta_param = self.shot_db[key]['beta'] + (self.shot_db[key]['shots'] - self.shot_db[key]['goals'])

                    # Calculate mean of beta distribution
                    prob = alpha / (alpha + beta_param)

                    # Calculate weight based on bin center distance
                    d_min, d_max, a_min, a_max, p_min, p_max = key
                    d_center = (d_min + d_max) / 2
                    a_center = (a_min + a_max) / 2
                    p_center = (p_min + p_max) / 2

                    # Inverse distance weighting
                    dist = np.sqrt(((distance - d_center)/120)**2 +
                                   ((angle - a_center)/90)**2 +
                                   ((pressure - p_center)/1.0)**2)
                    weight = 1 / (dist + 0.01)  # Add small constant to avoid division by zero

                    probs.append(prob)
                    weights.append(weight)

                # Normalized weighted average
                weights_sum = sum(weights)
                if weights_sum > 0:
                    weighted_prob = sum(p * w for p, w in zip(probs, weights)) / weights_sum
                else:
                    weighted_prob = 0.5  # Default if weights sum to zero

                # Heuristic model for comparison
                base_prob = max(0.01, min(0.7, 1 - (distance / 100))) * (1 - (angle / 180))
                skill_mod = (shooter_skill - 5) / 10
                pressure_mod = 1 - (pressure * 0.5)
                heuristic_prob = base_prob * (1 + skill_mod) * pressure_mod

                # Combine Bayesian and heuristic models
                # More weight to Bayesian model as we gather more data
                total_shots = sum(self.shot_db[key]['shots'] for key in nearest_bins)
                bayesian_weight = min(0.8, 0.3 + (total_shots / 1000) * 0.5)  # Max 80% weight for Bayesian

                final_prob = (weighted_prob * bayesian_weight) + (heuristic_prob * (1 - bayesian_weight))
                return min(0.95, max(0.01, final_prob))

        # If exact bin found, use it directly
        if bin_key:
            alpha = self.shot_db[bin_key]['alpha'] + self.shot_db[bin_key]['goals']
            beta_param = self.shot_db[bin_key]['beta'] + (self.shot_db[bin_key]['shots'] - self.shot_db[bin_key]['goals'])

            # Calculate mean of beta distribution with shooter skill adjustment
            skill_factor = (shooter_skill - 5) / 20  # Subtle adjustment for skill (-0.25 to +0.25)
            prob = (alpha / (alpha + beta_param)) * (1 + skill_factor)

            return min(0.95, max(0.01, prob))

        # Fallback to the original model if all else fails
        return self.calc_shot_prob(shooter, target_x, target_y, pressure)

    def update_shot_history(self, distance, angle, pressure, success):
        """Update shot history and Bayesian model with new data"""
        # First update the general shot history (parent class)
        super().update_shot_history(distance, angle, pressure, success)

        # Then update the Bayesian model
        outcome = 'goal' if success else 'miss'
        bin_key = self._find_bin(distance, angle, pressure)

        if bin_key and bin_key in self.shot_db:
            self.shot_db[bin_key]['shots'] += 1
            if outcome == 'goal':
                self.shot_db[bin_key]['goals'] += 1

    def execute_shot(self):
        """Execute a shot using the Bayesian model"""
        team = self.game_state.possession
        shooter_role = self.game_state.get_nearest_player(team, self.game_state.ball_x, self.game_state.ball_y)
        shooter = self.game_state.players[team][shooter_role]

        goal_x, goal_y = (120, 40) if team == "home" else (0, 40)
        pressure = self.calc_def_pressure(shooter.x_pos, shooter.y_pos)

        # Use the Bayesian model for shot probability
        shot_probability = self.calc_bayesian_shot_prob(shooter, goal_x, goal_y, pressure)

        # Calculate angle for reports/logs
        angle = abs(math.degrees(math.atan2(goal_y - shooter.y_pos, goal_x - shooter.x_pos)))
        distance = np.sqrt((goal_x - shooter.x_pos)**2 + (goal_y - shooter.y_pos)**2)

        print(f"Shot attempt by {team}'s {shooter_role} - Distance: {distance:.1f}m, Angle: {angle:.1f}°")
        print(f"Defensive Pressure: {pressure:.2f}, Shot Probability: {shot_probability:.3f}")

        shot_success = random.random() < shot_probability

        shooter_x = shooter.x_pos
        shooter_y = shooter.y_pos

        self.update_shot_history(distance, angle, pressure, shot_success, shooter_x, shooter_y)

        if shot_success:
            self.game_state.scoreline[team] += 1
            print(f"GOAL! {team.capitalize()} team scores!")
            print(f"New score: Home {self.game_state.scoreline['home']} - Away {self.game_state.scoreline['away']}")
            self.game_state.ball_x, self.game_state.ball_y = 60, 40
            self.game_state.possession = "away" if team == "home" else "home"
        else:
            if random.random() < 0.7:  # 70% chance goalkeeper saves
                print(f"Shot saved by the goalkeeper!")
                self.game_state.lose_possession()
            else:
                print(f"Shot misses the target!")
                # Determine if it's a goal kick or corner
                if random.random() < 0.5:
                    self.game_state.lose_possession()
                    if team == "home":
                        self.game_state.ball_x = 110
                    else:
                        self.game_state.ball_x = 10
                    self.game_state.ball_y = 40
                else:
                    # Corner kick
                    if team == "home":
                        self.game_state.ball_x = 115
                    else:
                        self.game_state.ball_x = 5
                    self.game_state.ball_y = random.choice([5, 75])

        return shot_success


class AdvancedBayesianShooting(BayesianShot):
    def __init__(self, game_state):
        super().__init__(game_state)
        self.shot_data = []

    def calculate_xg(self, shooter, x, y):
        """Calculate expected goals based on shot position and context"""
        # Get shooter skill
        shooting_skill = getattr(shooter, 'attributes', {}).get('shooting', 7)

        # Calculate distance to goal
        if self.game_state.possession == "home":
            goal_x, goal_y = 120, 40
        else:
            goal_x, goal_y = 0, 40

        distance = np.sqrt((x - goal_x)**2 + (y - goal_y)**2)

        # Calculate angle (in radians)
        dy = abs(y - goal_y)
        dx = abs(x - goal_x)
        angle = math.degrees(math.atan2(dy, dx))
        angle_difficulty = 1 - (angle / 90)  # 1 when straight on, 0 when from corner

        # Get defensive pressure
        defensive_pressure = self.calc_def_pressure(x, y)

        # Use our Bayesian model for xG
        xg = self.calc_bayesian_shot_prob(shooter, goal_x, goal_y, defensive_pressure)

        print(f"Shot xG: {xg:.3f} (distance: {distance:.1f}m, angle: {angle:.1f}°, angle factor: {angle_difficulty:.2f})")
        return xg

    def enhanced_execute_shot(self):
        """Execute a shot with enhanced xG model and outcome determination"""
        team = self.game_state.possession
        shooter_role = self.game_state.get_nearest_player(team, self.game_state.ball_x, self.game_state.ball_y)
        shooter = self.game_state.players[team][shooter_role]

        # Calculate xG
        xg = self.calculate_xg(shooter, self.game_state.ball_x, self.game_state.ball_y)

        shot_data = {
            'team': team,
            'shooter': shooter_role,
            'minute': getattr(self.game_state, 'time', 0),
            'xG': xg,
            'position': (self.game_state.ball_x, self.game_state.ball_y)
        }

        # Determine outcome
        outcome_roll = random.random()

        if outcome_roll < xg:
            self.game_state.scoreline[team] += 1 #Goal
            shot_data['outcome'] = 'goal'
            self.game_state.ball_x, self.game_state.ball_y = 60, 40
            self.game_state.possession = "away" if team == "home" else "home"
            print(f"GOAL! {team.capitalize()} scores! {shooter_role} finishes with {xg:.2f} xG")
        else:
            # Shot saved or missed
            if outcome_roll < xg + 0.4:  # 40% chance of save when shot doesn't score
                shot_data['outcome'] = 'saved'
                print(f"Shot saved! {shooter_role}'s attempt is stopped by the goalkeeper")
                self.game_state.lose_possession()
            else:
                shot_data['outcome'] = 'missed'
                print(f"Shot missed! {shooter_role}'s attempt goes wide")

                # Determine if corner or goal kick
                if random.random() < 0.4:  # 40% chance of corner
                    print("Corner kick awarded")
                    if team == "home":
                        self.game_state.ball_x = 118
                        self.game_state.ball_y = random.choice([2, 78])
                    else:
                        self.game_state.ball_x = 2
                        self.game_state.ball_y = random.choice([2, 78])
                else:
                    print("Goal kick")
                    self.game_state.lose_possession()
                    if team == "home":
                        self.game_state.ball_x = 2
                    else:
                        self.game_state.ball_x = 118
                    self.game_state.ball_y = 40

        # Update our Bayesian model
        distance = np.sqrt((shot_data['position'][0] - (120 if team == "home" else 0))**2 +
                          (shot_data['position'][1] - 40)**2)
        angle = abs(math.degrees(math.atan2(shot_data['position'][1] - 40,
                                           shot_data['position'][0] - (120 if team == "home" else 0))))
        pressure = self.calc_def_pressure(shot_data['position'][0], shot_data['position'][1])
        success = shot_data['outcome'] == 'goal'

        shooter_x = shooter.x_pos
        shooter_y = shooter.y_pos

        self.update_shot_history(distance, angle, pressure, success, shooter_x, shooter_y)
        self.shot_data.append(shot_data)

        return shot_data['outcome'] == 'goal'

# Event Class

In [531]:
class Event:
  def __init__(self, game_state):
        self.game_state = game_state
        self.passing_system = Pass(game_state)
        self.shot_system = Shot(game_state)

  def determine_event_type(self):
        """Determine the next event type based on game state"""
        team = self.game_state.possession
        team_style = self.game_state.team_style[team]
        ball_location = self.game_state.ball_location

        # Base probabilities dependent on play style
        style_probabilities = {
            "possession": {"pass": 0.75, "shot": 0.15, "lose_poss": 0.05, "tackle": 0.03, "foul": 0.02},
            "counter attack": {"pass": 0.60, "shot": 0.25, "lose_poss": 0.07, "tackle": 0.05, "foul": 0.03},
            "long ball": {"pass": 0.55, "shot": 0.20, "lose_poss": 0.10, "tackle": 0.10, "foul": 0.05}
        }

        # Adjust based on ball location
        location_modifiers = {
            "def_third": {"pass": 1.2, "shot": 0.1, "lose_poss": 1.2, "tackle": 1.5, "foul": 1.2},
            "mid": {"pass": 1.0, "shot": 0.3, "lose_poss": 1.0, "tackle": 1.0, "foul": 1.0},
            "off_third": {"pass": 0.8, "shot": 3.0, "lose_poss": 1.1, "tackle": 0.8, "foul": 0.9}
        }

        # Apply modifiers
        adjusted_probs = {}
        for event, base_prob in style_probabilities[team_style].items():
            adjusted_probs[event] = base_prob * location_modifiers[ball_location][event]

        # Normalize probabilities
        total = sum(adjusted_probs.values())
        for event in adjusted_probs:
            adjusted_probs[event] /= total

        # Special case: force higher shot probability in promising positions
        if ball_location == "off_third" and 90 <= self.game_state.ball_x <= 110 and 20 <= self.game_state.ball_y <= 60:
            shot_boost = 0.4  # Significant boost for shots in dangerous areas
            shot_prob = adjusted_probs["shot"] + shot_boost

            # Reduce other probabilities proportionally
            reduction_factor = (1 - shot_prob) / (1 - adjusted_probs["shot"])
            for event in adjusted_probs:
                if event != "shot":
                    adjusted_probs[event] *= reduction_factor
            adjusted_probs["shot"] = shot_prob

        # Convert to list of events and weights for random.choices
        events = list(adjusted_probs.keys())
        weights = list(adjusted_probs.values())

        selected_event = random.choices(events, weights=weights)[0]
        print(f"Selected event: {selected_event}")
        return selected_event

  def handle_event(self):
        """Handle the next game event"""
        event_type = self.determine_event_type()

        if event_type == "pass":
            self.passing_system.pass_execution()
        elif event_type == "shot":
            self.shot_system.execute_shot()
        elif event_type == "lose_poss":
            print("Team lost possession")
            self.game_state.lose_possession()
        elif event_type == "tackle":
            # Simplified tackle logic
            success = random.random() < 0.6  # 60% success rate
            if success:
                print("Successful tackle!")
                self.game_state.lose_possession()
            else:
                print("Failed tackle attempt")
        elif event_type == "foul":
            # Simplified foul logic
            print("Foul committed")
            self.game_state.lose_possession()

        # Update game time
        self.game_state.time += 1

        # Update all player positions based on new ball position
        self.game_state.player_movement()

        return event_type





# Event Probability function

In [532]:
def event_probability(team_play_style):
  probabilities_style = {    #obvs change numbers later based on stuff
      "possession": [0.60, 0.10, 0.10, 0.10, 0.10],  # High pass rate
        "counter attack": [0.40, 0.20, 0.10, 0.15, 0.15],  # More shots & fouls
        "long ball": [0.30, 0.15, 0.20, 0.25, 0.10]  # Direct play, more tackles
  }

  return probabilities_style[team_play_style]

## Get user changes

In [533]:
def get_user_changes(game_state):
  team = input("Enter team to change (home/away): ").lower()
  if team not in ['home','away']:
    print("Invalid team. No changes made.")
    return

  new_playstyle = input("Choose new play style (possession, counter attack,long ball): ").lower()
  new_formation = input("Choose new formation (4-3-3, 4-4-2, 3-5-2, 5-3-2, or leave blank to keep current): ")

  game_state.change_playstyle(team, new_playstyle if new_playstyle else None, new_formation if new_formation else None)


# Game Simulation

In [534]:
def run_game_simulation(num_events=20):
    """Run a full game simulation for specified number of events"""
    game_state = GameState()
    event_system = Event(game_state)
    all_events = []

    for i in range(num_events):
        print(f"\n--- Event {i+1} ---")
        event_type = event_system.handle_event()

        #if i % 5 == 0:
         # get_user_changes(game_state)

        # Record event data
        all_events.append({
            "time": game_state.time,
            "event_type": event_type,
            "ball_x": game_state.ball_x,
            "ball_y": game_state.ball_y,
            "possession": game_state.possession,
            "scoreline": game_state.scoreline.copy()
        })

        print(f"Time played: {game_state.time} minutes, Score: Home {game_state.scoreline['home']} - Away {game_state.scoreline['away']}")
        print(f"Team with possession: {game_state.possession.capitalize()}")
        print(f"Ball position: ({game_state.ball_x:.1f}, {game_state.ball_y:.1f}) - {game_state.ball_location}")

    return game_state, all_events



# Visualising the Game

In [535]:
def visualise_game(game_state, events):
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.set_title('Match Event Timeline', fontsize=14)
    ax.set_xlabel('Match Minute', fontsize=12)
    ax.set_ylabel('Event Type', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.7)

    event_types = {"pass": 1, "shot": 2, "goal": 3, "tackle": 4, "foul": 5}
    colors = {"pass": "blue", "shot": "orange", "goal": "green", "tackle": "red", "foul": "purple"}

    for event in events:
        if event['event_type'] in event_types:
            ax.scatter(event['time'], event_types[event['event_type']],
                       color=colors[event['event_type']], label=event['event_type'] if event['time'] == 0 else "")

    ax.set_yticks(list(event_types.values()))
    ax.set_yticklabels(list(event_types.keys()))

    plt.legend()
    plt.show()

  # Pitch Visualization
    fig, ax = plt.subplots(figsize=(10, 6))
    pitch = Pitch()
    pitch.draw(ax=ax)

    for event in events:
        if event['event_type'] == "pass":
            pitch.arrows(event['ball_x'], event['ball_y'],
                         event['ball_x'] + 5, event['ball_y'] + 5,
                         width=2, color='blue', ax=ax)
        elif event['event_type'] == "shot":
            pitch.scatter(event['ball_x'], event['ball_y'], color='orange', s=100, ax=ax)
        elif event['event_type'] == "goal":
            pitch.scatter(event['ball_x'], event['ball_y'], color='green', s=150, ax=ax)

    plt.show()



In [536]:
def draw_ball(x, y):
  """Draws the ball as a circle on the pitch."""
  ball = plt.Circle((ball_x, ball_y), radius=1, color='black')  # Adjust radius and color as needed
  return ball

In [537]:
def interactive_game_visualisation(game_state):
    """Visualizes the game using mplsoccer and matplotlib."""
    pitch = Pitch(pitch_type='statsbomb', pitch_color='#22312b', line_color='#c7d5cc')
    fig, ax = pitch.draw(figsize=(16, 11), constrained_layout=True, tight_layout=False)
    fig.set_facecolor("#22312b")

    # Extract player positions for initial scatter plot
    home_positions = np.array(list(game_state.player_pos['home'].values()))
    away_positions = np.array(list(game_state.player_pos['away'].values()))

    # Create scatter plots for home and away teams - no highlights
    scat_home = ax.scatter(home_positions[:, 0], home_positions[:, 1], marker='o', color='blue', s=200, zorder=2)
    scat_away = ax.scatter(away_positions[:, 0], away_positions[:, 1], marker='o', color='red', s=200, zorder=2)

    # Create a separate scatter for the ball with distinct appearance
    scat_ball = ax.scatter(game_state.ball_x, game_state.ball_y, marker='o', color='white', s=80, edgecolor='black', linewidth=1.5, zorder=4)

    # Initialize ball position history
    ball_history = [(game_state.ball_x, game_state.ball_y)]

    # Create a line for the ball trail with higher visibility
    ball_line, = ax.plot([game_state.ball_x], [game_state.ball_y], color='yellow', linestyle='-',
                         alpha=0.7, linewidth=2, zorder=3)

    # Create lines for player movement trails
    lines = []
    line_data = []
    for team in ["home", "away"]:
        for role, player in game_state.players[team].items():
            line, = ax.plot([], [], color='gray', linestyle='--', alpha=0.5, zorder=1)
            lines.append(line)
            line_data.append(player.position_history)
            player.position_history = [(player.x_pos, player.y_pos)]

    # Animation update function
    def update(frame):
        game_state.player_movement()
        game_state.update_ball_location()

        # Track ball position
        ball_history.append((game_state.ball_x, game_state.ball_y))

        # Update ball trail with all historical positions
        if len(ball_history) > 1:
            x_coords, y_coords = zip(*ball_history)
            ball_line.set_data(x_coords, y_coords)

        # Update scatter points with new player positions
        home_positions = np.array(list(game_state.get_player_pos()['home'].values()))
        away_positions = np.array(list(game_state.get_player_pos()['away'].values()))

        scat_home.set_offsets(home_positions)
        scat_away.set_offsets(away_positions)

        # Update ball position with clear visibility
        scat_ball.set_offsets(np.array([[game_state.ball_x, game_state.ball_y]]))

        # Update the data for the lines
        line_data = []
        for team in ["home", "away"]:
            for role, player in game_state.players[team].items():
                player.position_history.append((player.x_pos, player.y_pos))
                line_data.append(player.position_history)

        # Set the new data for the lines
        for line, data in zip(lines, line_data):
            if data:  # Check if there's data to unpack
                x_coords, y_coords = zip(*data)  # Extract x and y coordinates
                line.set_data(x_coords, y_coords)  # Update line data

        # Return a list containing all the artists that were updated
        return [scat_home, scat_away, scat_ball, ball_line] + lines

    # Create animation
    ani = FuncAnimation(fig, update, frames=100, interval=100, blit=True, repeat=False)

    # Display animation in Colab
    display(HTML(ani.to_jshtml()))

# Main Program

In [538]:
def main():
    final_game_state, events = run_game_simulation(40)
    visualise_game(final_game_state, events)
    game_state = GameState()
    interactive_game_visualisation(game_state)

    print("\nFinal game state:")
    print(f"Score: Home {final_game_state.scoreline['home']} - Away {final_game_state.scoreline['away']}")
    print(f"Possession: {final_game_state.possession.capitalize()}")
    print(f"Game time: {final_game_state.time} minutes")

In [539]:
if __name__ == "__main__":

    main()

No historical pass data found. Creating a new dataset.


KeyError: 'shooter_x'